In [1]:
import pandas as pd
import numpy as np
import math
import scipy
import os
import ivolatility as ivol
from datetime import datetime, timedelta
from scipy.stats import norm

### SPY Option Developments (for context):
- **February 2018**: Monday-expiring weekly SPY options were introduced
- **2022**: Additional weekday expirations added
- **Today**: SPY now has options expiring every single trading day of the week

So in 2008, if you wanted short-term SPY exposure, your only choice was the nearest monthly contract expiring on the third Friday. The weekly options revolution for SPY was still 2+ years away. Hence we will only replicate the risk reversal strategy with the options closet to 1-month of expiry in our backtest.

In [2]:
historical_rally_dates =['2008-10-13',
                        '2008-10-28',
                        '2009-03-23',
                        '2020-03-13',
                        '2020-03-24']

ticker = "SPY"
moneyness = 5

def get_eod_option_series(
    symbol: str,
    trade_date: str,
    dte_from: int,
    dte_to: int,
    moneyness_from: float,
    moneyness_to: float,
    call_put: str = None,
    combined: bool = False,
    save_path: str = None
) -> pd.DataFrame:
    """
    Retrieve end-of-day option data for a given symbol and trade date
    from the IVolatility API using the /equities/eod/stock-opts-by-param
    endpoint, filtered by DTE, moneyness, and option type.

    Parameters
    ----------
    symbol : str
        Ticker symbol (e.g., "SPY").
    trade_date : str
        The trading date in "YYYY-MM-DD" format.
    dte_from : int
        Minimum days to expiration.
    dte_to : int
        Maximum days to expiration.
    moneyness_from : float
        Minimum moneyness.
    moneyness_to : float
        Maximum moneyness.
    call_put : str, optional
        "C" for calls, "P" for puts. Ignored if combined=True.
    combined : bool, optional
        If True, return both calls and puts combined. Default is False.
    save_path : str, optional
        File path to save the returned DataFrame. If provided, the data will be
        written to a CSV file.

    Returns
    -------
    pd.DataFrame
        DataFrame containing filtered option series data.
    """

    # Load API key
    api_key = os.getenv("API_KEY")
    if not api_key:
        raise ValueError("Environment variable 'API_KEY' is not set.")

    # Authenticate
    ivol.setLoginParams(apiKey=api_key)

    # Prepare endpoint
    getMarketData = ivol.setMethod('/equities/eod/stock-opts-by-param')

    def fetch(cp):
        return getMarketData(
            symbol=symbol,
            tradeDate=trade_date,
            dteFrom=dte_from,
            dteTo=dte_to,
            moneynessFrom=moneyness_from,
            moneynessTo=moneyness_to,
            cp=cp
        )

    # If combined, pull both calls and puts
    if combined:
        df_call = fetch("C")
        df_put = fetch("P")

        df = pd.concat([df_call, df_put], ignore_index=True)
        df = df.sort_values(by=["price_strike", "call_put"]).reset_index(drop=True)

    else:
        if call_put not in ("C", "P"):
            raise ValueError("call_put must be 'C' or 'P' unless combined=True.")
        df = fetch(call_put)

    # Save to CSV if requested
    if save_path:
        df.to_csv(save_path, index=False)

    return df

def find_otm_options(df, target_otm=5, sort=True) -> pd.DataFrame:
    """
    Adds an 'absolute_change' column to the DataFrame based on how far
    df['calc_OTM'] is from a target OTM value (in percent difference).
    
    Parameters
    ----------
    df : pandas.DataFrame
        Must contain a 'calc_OTM' column.
    target_otm : float
        Target OTM value used for comparison.
    sort : bool, optional (default=True)
        If True, returns the DataFrame sorted by absolute_change.
    
    Returns
    -------
    pandas.DataFrame
        Modified DataFrame with a new 'absolute_change' column.
    """

    df = df.copy()

    df["absolute_change"] = (
        (df["calc_OTM"] - target_otm).abs() / target_otm * 100
    )

    if sort:
        df = df.sort_values("absolute_change", ascending=True)

    return df.head(2)


def add_trading_days(date_str, x):
    """
    Add `x` trading days (Mon–Fri) to a date string 'YYYY-MM-DD'.
    Returns a new date string in the same format.
    """
    date = datetime.strptime(date_str, "%Y-%m-%d")

    delta = 1 if x >= 0 else -1
    days_to_add = abs(x)

    while days_to_add > 0:
        date += timedelta(days=delta)
        if date.weekday() < 5:  # 0=Mon, 4=Fri
            days_to_add -= 1

    return date.strftime("%Y-%m-%d")


In [3]:
options_data_before = []
options_data_after = []

for i in range(len(historical_rally_dates)):
    
    options_symbol_list = []

    if i <= 2:# for options in 2010 and before
        dte_from = 25
        dte_to = 45
    else:
        dte_from = 30
        dte_to = 31

    print(historical_rally_dates[i])
    date_before = historical_rally_dates[i]
    trading_days = 17
    date_after = add_trading_days(date_before, trading_days)

    eod_option_chain = get_eod_option_series(
        symbol=ticker,
        trade_date=date_before,
        dte_from=dte_from,
        dte_to=dte_to,
        moneyness_from=moneyness - 1,
        moneyness_to=moneyness + 1,
        combined=True,
        save_path=None
    )

    otm_five_percent = find_otm_options(eod_option_chain)


    # Validate call/put presence
    if len(set(otm_five_percent["call_put"])) != 2:
        print(set(otm_five_percent["call_put"]))
        raise ValueError(
            "The list 'otm_five_percent' must contain exactly one call and one put option"
        )

    options_data_before.append(otm_five_percent)
    options_symbol_list += list(otm_five_percent['option_symbol'])
    #-------------------------------------------------------------------------

    dte_from=1
    dte_to=25


    eod_option_chain_after = get_eod_option_series(
        symbol=ticker,
        trade_date=date_after,
        dte_from=dte_from,
        dte_to=dte_to,
        moneyness_from=-100,
        moneyness_to=100,
        combined=True,
        save_path=None
    )

    otm_five_percent_after =  eod_option_chain_after[eod_option_chain_after['option_symbol'].isin(options_symbol_list)]

    # Validate call/put presence
    if len(set(otm_five_percent_after["call_put"])) != 2:
        raise ValueError(
            "The list 'otm_five_percent_after' must contain exactly one call and one put option"
        )

    options_data_after.append(otm_five_percent_after)


2008-10-13
2008-10-28
2009-03-23
2020-03-13
2020-03-24


In [4]:
df_list = []
for df_before,df_after in zip(options_data_before,options_data_after):
    df_list.append(df_before)
    df_list.append(df_after)

combined_df = pd.concat(df_list,ignore_index=True)
combined_df.to_csv('../data/Option/historical_options_prices_on_and_after_rally_days.csv', index=False)
combined_df 

,c_date,option_symbol,dte,stocks_id,expiration_date,call_put,price_strike,price_open,price_high,price_low,...,theta,vega,rho,Ask,Bid,underlying_price,calc_OTM,option_id,is_settlement,absolute_change
0,2008-10-13,SWGWR,40,627,2008-11-22,P,96.0,8.05,8.40,4.35,...,-0.079974,0.120172,-0.036274,4.50,4.35,101.35,5.28,None,0,5.6
1,2008-10-13,SWGKB,40,627,2008-11-22,C,106.0,2.70,4.00,2.21,...,-0.080062,0.129650,0.041189,4.35,4.15,101.35,4.59,None,0,8.2
2,2008-11-05,SWGWR,17,627,2008-11-22,P,96.0,2.60,4.41,2.23,...,-0.117984,0.082559,-0.020001,4.00,3.90,96.19,0.20,None,0,NaN
3,2008-11-05,SWGKB,17,627,2008-11-22,C,106.0,1.04,1.14,0.47,...,-0.052859,0.045356,0.005852,0.55,0.53,96.19,10.20,None,0,NaN
4,2008-10-28,SZCWK,25,627,2008-11-22,P,89.0,6.75,8.55,3.90,...,-0.115260,0.089995,-0.022160,4.15,3.95,93.76,5.08,None,0,1.6
5,2008-10-28,SWGKT,25,627,2008-11-22,C,98.0,1.85,3.85,1.35,...,-0.108802,0.095686,0.024054,3.85,3.65,93.76,4.52,None,0,9.6
6,2008-11-20,SZCWK,2,627,2008-11-22,P,89.0,8.90,13.75,6.74,...,-0.119975,0.001647,-0.001327,13.90,13.25,75.45,-17.96,None,0,NaN
7,2008-11-20,SWGKT,2,627,2008-11-22,C,98.0,0.01,0.03,0.01,...,-0.009023,0.000115,0.000002,0.02,0.01,75.45,29.89,None,0,NaN
8,2009-03-23,SZCPZ,26,627,2009-04-18,P,78.0,3.00,3.10,1.80,...,-0.065305,0.075398,-0.016752,1.94,1.89,82.22,5.13,None,0,2.6
9,2009-03-23,SZCDH,26,627,2009-04-18,C,86.0,0.74,1.72,0.62,...,-0.057741,0.078621,0.017863,1.69,1.65,82.22,4.60,None,0,8.0
